# 3-2. torch 자동미분


딥러닝 신경망을 학습할 때 가장 자주 사용되는 알고리즘은 역전파입니다. 이 알고리즘에서, 매개변수(모델 가중치)는 주어진 매개변수에 대한 손실 함수의 변화도(gradient)에 따라 조정됩니다. 이러한 변화도를 계산하기 위해 PyTorch에는 torch.autograd라고 불리는 자동 미분 엔진이 내장되어 있습니다. 이것은 모든 계산 그래프에 대한 변화도의 자동 계산을 지원합니다.

예를들어 자동미분을 설명해 보겠습니다.

y = W^3 + 2 * W^2 +7 이라는 공식이 있다면 W = 3 값에 대한 미분을 진행해 보겠습니다.

미분 공식으로 풀면 3 * W^2+2*W 이므로 W=3 이라면 3 * 3^2+2 * 2 * 3^**1 = 39가 결과로 나와야 합니다. 프로그램에서 자동미분으로 풀어 보겠습니다. 먼저 값이 3인 스칼라 텐서 W를 선언합니다. 선언 시, requires_grad = Ture 로 설정하면 이 텐서에 대한 기울기를 저장하게 됩니다. 수식도 정의해줍니다. 수식에 대한 미분을 backward() 메서드를 호출하여 진행합니다.



In [1]:
import torch

w = torch.tensor(3.0, requires_grad = True)
y = w**3 + 2*w**2 + 7
y.backward()
print(f'y를 w로 미분한 값 : {w.grad}')


y를 w로 미분한 값 : 39.0


입력 x, 매개변수 w와 b , 그리고 일부 손실 함수가 있는 가장 간단한 단일 계층 신경망을 가정하겠습니다. PyTorch에서는 다음과 같이 정의할 수 있습니다:



In [4]:
import torch

x = torch.ones(5)  # 입력 tensor
y = torch.zeros(3)  # 정답 tensor
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

이 신경망에서, w와 b는 최적화를 해야 하는 매개변수입니다. 따라서 이러한 변수들에 대한 손실 함수의 변화도를 계산할 수 있어야 합니다. 이를 위해서 해당 텐서에 requires_grad 속성을 설정합니다.

신경망에서 매개변수의 가중치를 최적화하려면 매개변수에 대한 손실함수의 도함수(derivative)를 계산해야 합니다. 이러한 도함수를 계산하기 위해, loss.backward() 를 호출한 다음 w.grad와 b.grad에서 값을 가져옵니다:



In [5]:
loss.backward()
print(w.grad)
print(b.grad)


tensor([[0.0016, 0.0042, 0.3314],
        [0.0016, 0.0042, 0.3314],
        [0.0016, 0.0042, 0.3314],
        [0.0016, 0.0042, 0.3314],
        [0.0016, 0.0042, 0.3314]])
tensor([0.0016, 0.0042, 0.3314])


다른 예제를 풀어 보겠습니다. 입력, 출력 그리고 가중치, 편향, 손실함수를 모두 임의로 만들고, 자동 미분을 사용해보겠습니다. 입력, 출력, 가중치, 편향을 모두 임의로 만들어줍니다.


In [9]:
x = torch.ones(5)  # 입력
y = torch.zeros(3)  # 출력

w = torch.randn(5, 3, requires_grad=True) # 가중치
b = torch.randn(3, requires_grad=True) # 편향

식을 정의하고 손실 함수는 MSE 를 사용하겠습니다.



In [12]:
import torch.nn as nn
z = torch.matmul(x, w) + b
loss_fn = nn.MSELoss() # 손실 함수

손실함수 값을 계산하고, backward()를 호출해 기울기를 계산합니다.



In [13]:
loss = loss_fn(z, y)
loss.backward()


결과는 아래와 같습니다.



In [14]:
print(f'loss를 w로 미분한 값 :\n {w.grad}')
print(f'loss를 b로 미분한 값 : {b.grad}')


loss를 w로 미분한 값 :
 tensor([[-1.7398, -0.4275,  1.1060],
        [-1.7398, -0.4275,  1.1060],
        [-1.7398, -0.4275,  1.1060],
        [-1.7398, -0.4275,  1.1060],
        [-1.7398, -0.4275,  1.1060]])
loss를 b로 미분한 값 : tensor([-1.7398, -0.4275,  1.1060])


requires_grad=True인 모든 텐서들은 연산 기록을 추적하고 변화도 계산을 지원합니다. 그러나 모델을 학습한 뒤 입력 데이터를 단순히 적용하기만 하는 경우와 같이 순전파 연산만 필요한 경우에는, 이러한 추적이나 지원이 필요없을 수 있습니다. 연산 코드를 torch.no_grad() 블록으로 둘러싸서 연산 추적을 멈출 수 있습니다:



In [15]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)


True
False


동일한 결과를 얻는 다른 방법은 텐서에 detach() 메소드를 사용하는 것입니다:



In [16]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)


False


변화도 추적을 멈춰야 하는 이유들은 다음과 같습니다: 신경망의 일부 매개변수를 고정된 매개변수(frozen parameter)로 표시합니다. 이는 사전 학습된 신경망을 미세조정 할 때 매우 일반적인 시나리오입니다. 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상됩니다.

